## EE 242 Lab 1a – Modifying Signals - Sampling and Amplifying

**Team AAO06: Aidan Frondozo, Kiet To, Chinmay Murthy, Andrew Tat**

This lab has 2 exercises to be completed as a team. Each should be given a separate code cell in your Notebook, followed by a markdown cell with report discussion. Your notebook should start with a markdown title and overview cell, which should be followed by an import cell that has the import statements for all assignments. For this assignment, you will need to import: numpy, the wavfile package from scipy.io, and matplotlib.pyplot.  

In [2]:
# We'll refer to this as the "import cell." Every module you import should be imported here.
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.io import wavfile as wav
from IPython.display import Audio

## Summary

In this lab, you will work through a series of exercises to introduce you to working with audio signals and explore the impact of different amplitude and time operations on signals.  This is a two-week lab.  You should plan on completing the first 2 assignments in the first week.

## Lab 1a turn in checklist

•	Lab 1a Jupyter notebook with code for the 2 exercises assignment in separate cells. Each assignment cell should contain markdown cells (same as lab overview cells) for the responses to lab report questions. Include your lab members’ names at the top of the notebook.

**Please submit the report as PDF**




## Assignment 1 -- Working with sound files

Start a new cell following the guidelines in the Lab 1a template, dividing it into Parts A-C.

**A.**  Download the train.wav sound file provided. Read in the file saving the audio vector and sampling frequency in variables $x_1(t)$ and $f_{s_1}$, respectively. Print the sampling rate $f_{s_1}$ (which should be 32kHz) and the shape of $x_1(t)$, which will tell you the length and number of channels.  

**B.**  Write out two new versions of the file in wav format using different sampling rates: $f_{s_2}=f_{s_1}/2$ (16 kHz) and $f_{s_3}=1.5f_{s_1}$.  Basically create $x_2(t)=x_1(t/2)$ and $x_3(t)=x_1(2t)$

**C.**  Read in the three different versions of the train sound file and play each one.   ($x_1(t), x_2(t), x_3(t)$, don't change the $f_s$ while playing)

In [3]:
# Assignment 1 - Time Scaling Function
# Start with a comment section that explains what the input and output variables are, e.g.
# x: input signal vector
# fs: sampling rate (in Hz)
# a: scaling parameter. This has to be a decimal value for as_integer_ratio to work. 
# So, explicitly casting it into a float or a double or any fractional data type will help.
# returns t: time samples vector corresponding to y: scaled signal

# Resamples the given signal using a zero-phase low-pass FIR filter
def timescale(x, fs, a):
    [n, d] = (np.double(a)).as_integer_ratio()
    y = sig.resample_poly(x,d,n)
    t = np.arange(0,len(y),1)*(1.0/fs)
    return y,t


# Resamples the given signal via linear interpolation
def linear_timescale(x, fs, a):
    [n, d] = (np.double(a)).as_integer_ratio()
    
    input_t = np.arange(0,len(x),1)*(1.0/fs) # input t values given sample rate
    up_t = np.arange(0,len(x)*d,1)*(1.0/fs/d) # upsampled t values
    up_y = np.interp(up_t, input_t, x) # upsampled signal values
    t = up_t[::n] # downsample by selecting every nth element
    y = up_y[::n] # downsample by selecting every nth element
    
    return y,t


In [9]:
# Assignment 1 - Playing and Plotting Time Scaled Audio Files 

# Part A
# 1. Read the train.wav file
file = wav.read('./train32.wav')


# 2. Print the sampling rate and shape of signal
fs = file[0]
x1 = np.multiply(file[1], 1.0)

print('Sampling Rate: {}Hz'.format(fs))
print('Shape: {} => {} channel(s)'.format(file[1].shape, len(file[1].shape)))



# Part B
# 1. Use both timescale and linear_timescale functions above to resample the signal using value of "a" as 0.5 and 2.

# 2. Store the files as train_(high/low)_poly.wav and train_(high/low)_linear.wav (Don't change the f_s while storing the files)
def write(name, data):
    wav.write(name, fs, data.astype('int16'))

    
y, t = timescale(x1, fs, 0.5)
write('train_low_poly.wav', y)

y, t = linear_timescale(x1, fs, 0.5)
write('train_low_linear.wav', y)

y, t = timescale(x1, fs, 2)
write('train_high_poly.wav', y)

y, t = linear_timescale(x1, fs, 2)
write('train_high_linear.wav', y)




# Part C
# 1. Play the files
def play_file(name):
    print('\n'+name+":")
    display(Audio(filename=name))

play_file('train_low_poly.wav')
play_file('train_low_linear.wav')
play_file('train_high_poly.wav')
play_file('train_high_linear.wav')


Sampling Rate: 32000Hz
Shape: (50313,) => 1 channel(s)

train_low_poly.wav:



train_low_linear.wav:



train_high_poly.wav:



train_high_linear.wav:


###  Discussion

Comment on how the audio changes when the incorrect sampling frequency is used.

Similar to playing back the same signal at a different sampling rate, changing the sampling frequency stretches or shrinks the signal time-wise, meaning it changes the length and pitch at the same time.
When samples are taken twice as far apart and played back at the same sample rate, the length of the signal is halved and the frequencies are doubled (raised by an octave). 
When samples are taken half as far apart and played back at the same sample rate, the length of the signal is doubled and the frequencies are halved (lowered by an octave). 

## Assignment 2 -- Amplitude Operations on Signals

Again, following the guidelines in the Lab 1 template, start a new cell and write a script to meet the following specifications. This assignment will have three parts, A-C, each of which should be indicated with comments.  

**A.1.** Create a discrete time signal $s_1(t)$ that is the same length as $x_1(t)$ and has value 1 for t=[0,0.5] and value 0.2 for $t>0.5$.  

You can use the command below where $len$ is the length of $x_1(t)$ and $n_0$ is the index corresponding to t=0.5, 
s1 = np.concatenate((np.ones(n0),0.2*np.ones(len(x1)-n0)) 

**A.2.** Multiply x1 with s1 to create v1. Save this signal to a wav file. 

**B.1** Create a discrete-time decaying ramp signal r1, that is the same length as x1. The signal should have value 1 at time 0 and linearly decay to value 0.  (Hint: use numpy.arange.) 

**B.2** Multiply x1 with r1 to create v2. Save this signal to a wav file.  

**C.**  Read in v1 and v2 and play the two different modifications together with the original, to verify that the volume of the second whistle is reduced. 

Ensure to play all the signals for the TAs

In [7]:
# Assignment 2 - Amplitude Operations on Signals

# Part A
# 1. Create a signal s1 which 1 from 0s to 0.5s and 0.2 from 0.5s onward
n0 = int(0.5 * fs)
s1 = np.concatenate((np.ones(n0),0.2*np.ones(len(x1)-n0)))


# 2. Multiply s1 with x1 sample by sample to create v1
v1 = s1*x1


# Part B
# 1. Create a signal r1 which goes down linearly from 1 to 0 across the length of the signal.
r1 = np.linspace(1, 0, len(x1))


# 2. Multiply r1 with x1 sample by sample to create v2
v2 = r1 * x1


# Part C
# 1. Store and play v1 and v2
write('v1.wav', v1)
write('v2.wav', v2)
play_file('v1.wav')
play_file('v2.wav')



v1.wav:



v2.wav:


###  Discussion

Discuss the differences that the two modifications have on the signal. What would happen if you defined s1 to take value 2 for the [0,0.5] range? If you wanted a smooth but faster decay in amplitude, what signal might you use?

The first modification only ducks the amplitude down for the second part of the signal, whereas the second modification gradually brings the amplitude to 0 throughout the course of the signal. Therefore, the first signal sounds to the user like it says at the same volume except for the jump in the middle, while the second is audibly "fading away".
If s2 was 0.2 from [0,0,5] and 1 for the rest, the first part of the signal would be softer while the rest would be normal volume. If we wanted v2 to have a faster but smooth decay, instead of varying amplitude linearly, we could use a different function with a more drastic initial drop, like an exponential or polynomial decay function.